In [2]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [37]:
data = pd.read_csv('OV2.csv', on_bad_lines='skip', sep=';')
data = pd.DataFrame(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15207 entries, 0 to 15206
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              15207 non-null  object 
 1   Streams           15207 non-null  int64  
 2   Viewers           15207 non-null  int64  
 3   Dominant_streams  15207 non-null  int64  
 4   Dominant_ratio    15207 non-null  float64
 5   Tags              15207 non-null  object 
 6   Name              15207 non-null  object 
 7   Title             15207 non-null  object 
 8   Beginning         15207 non-null  object 
 9   Unnamed: 9        94 non-null     object 
 10  Unnamed: 10       46 non-null     object 
dtypes: float64(1), int64(3), object(7)
memory usage: 1.3+ MB


In [38]:
data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y %H:%M')
data['Hour'] = data['Date'].dt.hour
data['Minute'] = data['Date'].dt.hour
data['Weekday'] = data['Date'].dt.weekday
data['Moving_streams'] = data['Streams'].rolling(10, min_periods=1).mean()

data.head()

,Date,Streams,Viewers,Dominant_streams,Dominant_ratio,Tags,Name,Title,Beginning,Unnamed: 9,Unnamed: 10,Hour,Minute,Weekday,Moving_streams
0,2023-03-11 12:06:00,1,5,1,1.0,"[['Polski', 'DropyWłączone', 'Depresja', 'Hand...",['dariocosta'],"['1W - 1L | Gold 2 dps | Z rana Overek, a wiec...",['2023-03-11T10:27:52Z'],NaN,NaN,12,12,5,1.0
1,2023-03-11 12:08:00,1,5,1,1.0,"[['Polski', 'DropyWłączone', 'Depresja', 'Hand...",['dariocosta'],"['1W - 1L | Gold 2 dps | Z rana Overek, a wiec...",['2023-03-11T10:27:52Z'],NaN,NaN,12,12,5,1.0
2,2023-03-11 12:10:00,1,6,1,1.0,"[['Polski', 'DropyWłączone', 'Depresja', 'Hand...",['dariocosta'],"['1W - 1L | Gold 2 dps | Z rana Overek, a wiec...",['2023-03-11T10:27:52Z'],NaN,NaN,12,12,5,1.0
3,2023-03-11 12:12:00,1,6,1,1.0,"[['Polski', 'DropyWłączone', 'Depresja', 'Hand...",['dariocosta'],"['1W - 1L | Gold 2 dps | Z rana Overek, a wiec...",['2023-03-11T10:27:52Z'],NaN,NaN,12,12,5,1.0
4,2023-03-11 12:14:00,1,6,1,1.0,"[['Polski', 'DropyWłączone', 'Depresja', 'Hand...",['dariocosta'],"['1W - 1L | Gold 2 dps | Z rana Overek, a wiec...",['2023-03-11T10:27:52Z'],NaN,NaN,12,12,5,1.0


In [39]:
X = data[['Hour','Minute','Weekday']]
#y = data[['Streams','Dominant_streams','Viewers', 'Dominant_ratio']]

y = data[['Moving_streams']]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=False)
y_test = y_test.reset_index()

In [41]:

xgb_model = xgb.XGBRegressor(n_estimators = 1000)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

#For measuring accuracy 
y_pred = y_pred.astype(int)
y_test['Moving_streams'] = y_test['Moving_streams'].astype(int)



wynik = 0
wynik_list =[]
#For bigger games
# for i in range(len(y_test)):
#     temp = max(y_pred[i],y_test['Moving_streams'][i]) / min(y_pred[i],y_test['Moving_streams'][i])
#     temp = ((temp * 100) - 100) * 10
#     temp = 100 - temp
#     if temp > 0:
#         wynik += temp/100
#         wynik_list.append(temp)
#     else:
#         wynik += 0
#         wynik_list.append(0)

for i in range(len(y_test)):
    if y_test['Moving_streams'][i] == y_pred[i]:
        wynik += 1
        wynik_list.append(100)
    elif y_pred[i] == y_test['Moving_streams'][i] + 1 or  y_pred[i] == y_test['Moving_streams'][i] - 1:
        wynik += 0.5
        wynik_list.append(50)
    else:
        wynik += 0 
        wynik_list.append(0)

# For bigger games
# df = pd.DataFrame(y_pred)
# df['should'] = y_test['Moving_streams']
# df['result'] = wynik_list
# df.columns = ['is','should','result']
# display(df)

print(wynik/len(y_test))

df = pd.DataFrame(y_pred)
df['should'] = y_test['Moving_streams']
df['result'] = wynik_list
df.columns = ['is','should','result']
display(df)



0.46583442838370565


,is,should,result
0,4,6,0
1,4,6,0
2,4,6,0
3,4,6,0
4,4,6,0
...,...,...,...
756,5,4,50
757,5,4,50
758,5,4,50
759,5,4,50


In [60]:

new_data = pd.DataFrame({'Hour': [17], 'Minute': [0], 'Weekday': [0]})
prediction = xgb_model.predict(new_data)

print('Score: ', prediction)

Score:  [7.037115]


In [27]:
xgb_model.feature_importances_

array([0.91818583, 0.        , 0.08181416], dtype=float32)

In [70]:
from sklearn.ensemble import IsolationForest

X = data[['Hour','Minute','Weekday','Streams']]

isof = IsolationForest(n_estimators=1000, contamination=0.1)
isof.fit(X)

new_data = pd.DataFrame({'Hour': [17], 'Minute': [12], 'Weekday': [0], 'Streams': [11]})
display(new_data)
prediction = isof.predict(new_data)

print('Wynik: ', prediction)


c:\Users\tymot\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Hour,Minute,Weekday,Streams
0,17,12,0,11


Wynik:  [-1]
